In [ ]:
import requests

url = "https://www.openml.org/data/download/22102255/dataset"
r = requests.get(url, allow_redirects=True)

In [ ]:
with open("dataset.txt", "wb") as f:
    f.write(r.content)

In [ ]:
import pandas as pd

In [ ]:
data = []

with open("dataset.txt", "r") as f:
    for line in f.read().split("\n"):
        if line.startswith("@") or line.startswith("%") or line == "":
            continue
        data.append(line)

In [ ]:
columns = []

with open("dataset.txt", "r") as f:
    for line in f.read().split("\n"):
        if line.startswith("@ATTRIBUTE"):
            columns.append(line.split(" ")[1])

In [ ]:
with open("df.csv", "w") as f:
    f.write(",".join(columns))
    f.write("\n")
    f.write("\n".join(data))

In [ ]:
df = pd.read_csv("df.csv")
df.columns = columns

In [ ]:
df

In [ ]:
df["t_win"] = df.round_winner.astype("category").cat.codes
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numeric_columns = df.select_dtypes(include=['number', bool])
correlations = numeric_columns.corr()

print(correlations['t_win'].apply(abs).sort_values(ascending=False).iloc[:25])

selected_columns = []

for col in columns+['t_win']:
    try:
        if abs(correlations[col]['t_win']) > 0.15:
            selected_columns.append(col)
    except KeyError:
        pass

df_selected = df[selected_columns]

In [ ]:
df_selected

In [ ]:
print(len(df_selected.columns))

In [ ]:
print(correlations['bomb_planted']['t_win'])
#fix
plt.figure(figsize=(18, 12))
sns.heatmap(df_selected.corr().sort_values(by='t_win'), annot=True, cmap="YlGnBu")

In [ ]:
df_selected.hist(figsize=(18, 12))
plt.savefig("histo.png")

In [ ]:
df_selected.info()

In [ ]:
df_selected.describe()

In [ ]:
from sklearn.model_selection import train_test_split

X, y = df_selected.drop(["t_win"], axis=1), df_selected["t_win"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier()

knn.fit(X_train_scaled, y_train)

In [ ]:
knn.score(X_test_scaled, y_test)


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    "n_neighbors": list(range(5, 17, 2)),
    "weights": ["uniform", "distance"]
}

knn = KNeighborsClassifier(n_jobs=-1)

clf = RandomizedSearchCV(knn, param_grid, n_jobs=-1, n_iter=3, verbose=2, cv=3)
clf.fit(X_train_scaled, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_estimator_.score(X_test_scaled, y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_jobs=-1)

forest.fit(X_train_scaled, y_train)

In [ ]:
forest.score(X_test_scaled, y_test)

In [ ]:
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(20,)))
model.add(keras.layers.Dense(200, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5)

X_train_scaled_train, X_valid, y_train_train, y_valid = train_test_split(X_train_scaled, y_train, test_size=0.15)


model.fit(X_train_scaled_train, y_train_train, epochs=30, callbacks=[early_stopping_cb], validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_test_scaled, y_test)

In [ ]:
#the forest was the highest of all the models we tried .